In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz
from utils import sample_PLN
import scipy.linalg as SLA
import torch.linalg as TLA
import scipy
from fastPLN import fastPLN
from scipy.special import factorial
import math
import torch 
from pandas import read_csv

device :  cpu


In [2]:
%load_ext autoreload

%autoreload 2

$$
\begin{aligned}
W_{i} & \sim \mathcal{N}\left(0, I_{q}\right), \text { iid, } \quad i=1, \ldots, n \\
Z_{i} &=\beta \mathbf{x}_{i}+\mathbf{C} W_{i}, \quad i \in 1, \ldots, n \\
Y_{i j} \mid Z_{i j} & \sim \mathcal{P}\left(\exp \left(o_{i j}+Z_{i j}\right)\right)
\end{aligned}
$$

We want to maximize the log likelihood with respect to $\theta$ : 


$$\max _{\theta} P_{\theta}(Y)$$

But we need to integrate out $W$ in order to compute the quantity inside the max : 

$$
\begin{aligned}
P_{\theta}\left(Y_{i}\right) &=\int P_{\theta}\left(Y_{i}, W\right) d W \\
&=\int P_{\theta}\left(Y_{i} \mid W\right) p(W) d W
\end{aligned}
$$

This integral being untractable, we are going to approximate it with Monte Carlo methods : 

$$
\int p_{\theta}\left(Y_{i} \mid W\right) p(W) d W \approx \frac{1}{K} \sum_{k = 1 }^Kp_{\theta}\left(Y_{i} \mid W_{i,k}\right) p\left(W_{i,k}\right)
$$
$$W_{i,k} \sim \mathcal N (0, I_q)$$

The larger the $K$ the better the approximation.  

Let's compute $p_{\theta}\left(Y_{i} \mid W_{i,k}\right) p\left(W_{i,k}\right)$. 


First, 

$$
P\left(W_{i,k}\right)=\frac{1}{\sqrt{2 \pi}} \exp \left(-\frac{1}{2}\left\|W_{i,k}\right\|_{2}^{2}\right)
$$ 

Then, 

$$
\begin{aligned}
P_{\theta}\left(Y_{i} \mid W_{i,k}\right) &=\prod_{j=1}^{p} p_{\theta}\left(Y_{i j} \mid W_{i,k}\right) \\
&=\prod_{j=1}^{P} \frac{1}{Y_{i j} !} \exp \left(-\exp \left(o_{i j}+z_{i j}\right)\right) \exp \left(o_{i j}+z_{i j}\right)^{Y_{ij}}
\end{aligned}
$$
$$
Z_{i j}=x_{i}^{\top} \beta_{j}+W_{i,k}^{\top} C_{j}
$$

For numerical purposes, we may want to use a logarithmic scale and apply the exponential function after. Indeed, $Y_{ij}$ can go up to a thousand, and computing this factorial would give infinite values. 

$$
\log p_{\theta}\left(Y_{i} \mid W_{i,k}\right)=\sum_{j=1}^{p} - \ln \left(Y_{i j} !\right)-\exp \left(0_{i j}+Z_{i j}\right)+Y_{i j}\left(o_{i j}+Z_{i j}\right)
$$

We are going tu use the Stirling Formula in order to compute the log of the factorial, to avoid computing directly the factorial.  

We now need to compute the gradients. Since 

$$
\nabla_{\theta} p_{\theta}\left(Y_{i} \mid W_{i,k}\right)=p_{\theta}\left(Y_{i}| W_{i,k}\right) \nabla_{\theta} \log p_{\theta}\left(Y_{i} \mid W_{i,k}\right)
$$

We get : 

$$
\nabla_{\beta} p_{\theta}\left(Y_{i} \mid W_{i,k}\right)=p_{\theta}\left(Y_{i} \mid W_{i,k}\right)\left[-x_{i}^{\top} \exp \left(0_{i}+x_{i}^{\top} \beta+W_{i,k}^{\top} C\right)+ x_{i}^{\top}Y_i\right]
$$
This is if we take only one sample $Y_i$. If we take the whole dataset (or a mini-batch), we get (writed in matrix form) :

$$
\nabla_{\beta} p_{\theta}\left(Y \mid W_k\right)=p_{\theta}\left(Y \mid W_k\right)\left[-X^{\top} \exp \left(0+X^{\top} \beta+W_k^{\top} C\right)+ X^{\top}Y\right]
$$

sizes : 

$ Y : (n,p)$ 

$O : (n,p)$ 

$C :  (p,q)$ 

covariates ($x$) : $(n,d)$

$\beta : (d,p)$

In [3]:
def build_block_Sigma(p,k): 
    '''
    build a matrix per block of size (p,p). There will be k+1 blocks of size p//k.
    The first k ones will be the same size. The last one will be smaller (size (0,0) if k%p = 0)
    '''
    
    np.random.seed(0)
    alea = np.random.randn(k+1)**2+1# will multiply each block by some random quantities 
    Sigma = np.zeros((p,p))
    block_size,last_block_size = p//k, p%k
    for i in range(k): 
        Sigma[i*block_size : (i+1)*block_size ,i*block_size : (i+1)*block_size] = alea[i]*0.95**np.arange(block_size)
    if last_block_size >0 :
        Sigma[-last_block_size:,-last_block_size:] = alea[k]*toeplitz(0.98**np.arange(last_block_size))
    return Sigma+0.1*toeplitz(0.95**np.arange(p))


def C_from_Sigma(Sigma,q): 
    w,v = SLA.eigh(Sigma)
    C_reduct = v[:,-q:]@np.diag(np.sqrt(w[-q:]))
    return C_reduct

In [14]:
d = 4 # nb of cavariates
n = 2000; p = 20
q = 10

In [15]:
#torch.manual_seed(0)
true_Sigma = torch.from_numpy(build_block_Sigma(p,8))
true_C = torch.from_numpy(C_from_Sigma(true_Sigma, q))
true_beta =torch.randn((d, p))

covariates = torch.randn((n,d))
O =  1+torch.zeros((n,p))
sample_model = sample_PLN()
Y_sampled, Z_sampled  = sample_model.sample(true_Sigma,true_beta, O, covariates)
Y_sampled = torch.from_numpy(Y_sampled)

In [16]:
def log_stirling(n_):
    '''
    this function computes log(n!) even for n large. We use the Stirling formula to avoid 
    numerical infinite values of n!. It can also take tensors.
    
    args : 
         n_ : tensor. 
    return : an approximation of log(n!)
    '''
    n = torch.clone(n_) #clone the tensor by precaution
    n+= (n==0) # replace the 0 with 1. It changes anything since 0! = 1! 
    return torch.log(torch.sqrt(2*np.pi*n))+n*torch.log(n/math.exp(1)) #Stirling formula

In [17]:
class MC_PLNPCA(): 
    
    def __init__(self,q, batch_size): 
        self.q = q
        self.batch_size = batch_size 
        pass
    
    def init_data(self, Y,O,covariates): 
        '''
        Initialise some usefuls variables given the data. 
        We also initialise C and beta. 
        '''
        
        self.Y = Y 
        self.covariates = covariates 
        self.O = O 
        self.n = Y.shape[0] 
        self.p = Y.shape[1]
        self.d = self.covariates.shape[1]
        noise = torch.randn(self.p) 
        self.Sigma =  (torch.diag(noise**2)+ 1e-1)
        self.C = torch.clone(torch.from_numpy(C_from_Sigma(self.Sigma,self.q)))
        self.beta = torch.randn(self.d,self.p)
        self.beta.requires_grad_(True)
        #self.beta = torch.clone(true_beta)
        #self.C = torch.clone(true_C)
    
    def single_likelihood(self,i,acc): 
        '''
        computes the likelihood of a single point. Useless since We can compute it efficiently
        with batch_likelihood(). This is more a beta version. 
        '''
        N_iter = int(1/acc)
        Y_i = self.Y[i,:]
        x_i = self.covariates[i,:]
        O_i = self.O[i,:]
        E = 0
        for _ in range(N_iter):
            W = torch.randn(self.q)
            log_fact = torch.sum(log_stirling(Y_i))
            norm_W = 1/2*torch.norm(W)**2
            poiss_like = - torch.sum(torch.exp(self.O[i,:]+self.beta.T@self.covariates[i,:]+ self.C@W)) 
            poiss_like += torch.sum((self.O[i,:]+self.beta.T@self.covariates[i,:]+ self.C@W)*self.Y[i,:])
            E+= torch.exp(-log_fact -norm_W + poiss_like)
        return E/N_iter 
    def get_Sigma(self):
        '''
        simple function to get Sigma
        '''
        return self.C@(self.C.T)
    
    def batch_likelihood(self,Y_batch,covariates_batch, O_batch, W, somme = True ): 
        '''
        computes the approximation of the likelihood of a batch. 
        
        args : 
                'Y_batch' : tensor of size(batch_size, p)
                'covariates_batch' : tensor of size(batch_size, d)
                'O_batch' : tensor of size(batch_size, p)
                'acc' : float. the accuracy you want. The lower the accuracy, the lower the algorithm. 
                        we will sampThe size of tensor a (1000) must match the size of tensor b (20) at non-singleton dimension 2les 1/acc times. 
        returns : 
                the approximation of the likelihood. 
        ''' 
        last_dim = len(W.shape)-1
        #if last_dim >1 : 
        #    N_samples = W.shape[0] # number of samples of W 
        #else : N_samples = 1
        N_samples = W.shape[0]
        E_batch = 0
        Z = covariates_batch@self.beta + W@(self.C.T)
        norm_W = TLA.norm(W, dim = last_dim)
        log_fact =  torch.sum(log_stirling(Y_batch), axis = 1) # the factorial term 
        poiss_like =  - torch.sum(torch.exp(O_batch+Z), axis = last_dim) # first term of the poisson likelihood
                                                                         #the normalising term with the exponential 
        poiss_like += torch.sum((O_batch+Z)*Y_batch, axis = last_dim)    # second ter of the poisson likelihood
        if somme : 
            # If we want the true likelihood
            # We first take the exponential of the sum of the logs and then divide by the Number of samples we took.  
            return torch.sum(torch.exp(-log_fact -1/2*norm_W+poiss_like))/N_samples 
        #for some purposes, we may want only the exponential and not the sum
        else : return torch.exp(-log_fact -1/2*norm_W+poiss_like)/N_samples


    def get_batch(self,batch_size): 
        '''
        get the batches required to do a  minibatch gradient ascent.  
        
        args : 
                'batch_size' int.  the batch size you want. 
                
        returns : a generator. Will generate n/batch_size samples of size batch_size (except the last one 
                    since the rest of the division is not always an integer)
                    
        '''
        np.random.seed(0)
        indices = np.arange(self.n)
        np.random.shuffle(indices)
        self.batch_size = batch_size 
        nb_full_batch, last_batch_size  = self.n//self.batch_size, self.n % self.batch_size  
        for i in range(nb_full_batch): 
            yield   (self.Y[indices[i*self.batch_size: (i+1)*self.batch_size]], 
                    self.covariates[indices[i*self.batch_size: (i+1)*self.batch_size]],
                    self.O[indices[i*self.batch_size: (i+1)*self.batch_size]]) 
                        
                  
        if last_batch_size != 0 : 
            self.batch_size = last_batch_size
            yield   (self.Y[indices[-last_batch_size:]], 
                    self.covariates[indices[-last_batch_size:]],
                    self.O[indices[-last_batch_size:]])
        
    def compute_likelihood(self, acc): 
        likelihood = 0
        N_samples = int(1/acc)
        W = torch.randn(N_samples, self.n, self.q)
        likelihood +=  self.batch_likelihood(self.Y,self.covariates, self.O,W)
        return likelihood
    def batch_grad_beta(self,Y_b,covariates_b, O_b, acc): 
        grad = 0 
        N_samples = int(1/acc)
        W = torch.randn(N_samples,self.batch_size, self.q) 
        
        log_like = torch.sum(self.batch_likelihood(Y_b,covariates_b, O_b,W, somme = False), axis = 1)
        exp_term  = -torch.exp(O_b +covariates_b@self.beta + W@(self.C.T))
        grad =  torch.sum(torch.multiply(log_like.reshape(-1,1,1),((covariates_b.T)@(exp_term + Y_b))), axis = 0)
        # the for loop here does the same, just a sanity check
        '''
        for i in range(N_samples): 
            log_like = self.batch_likelihood(Y_b,covariates_b, O_b,W[i])/N_iter
            exp_term  = -torch.exp(O_b +covariates_b@self.beta + W[i]@(self.C.T))
            grad +=  log_like*(covariates_b.T)@(exp_term + Y_b)
        '''
        return grad/N_samples
    
    def fit(self, N_iter, acc): 
        optim = torch.optim.Rprop([self.beta], lr = 0.3)
        for i in range(N_iter):
            for Y_b, covariates_b, O_b in model.get_batch(self.batch_size):
                optim.zero_grad()
                grad_beta = self.batch_grad_beta(Y_b,covariates_b, O_b,acc)
                self.beta.grad = -grad_beta/torch.norm(grad_beta)
                optim.step()
                print('MSE : ', torch.mean((self.beta-true_beta)**2))
                #print('beta : ', self.beta)
            print('likelihood : ', self.compute_likelihood(acc))
            

$$
\nabla_{\beta} p_{\theta}\left(Y \mid W_k\right)=p_{\theta}\left(Y \mid W_k\right)\left[-X^{\top} \exp \left(0+X^{\top} \beta+W_k^{\top} C\right)+ X^{\top}Y\right]
$$

In [18]:
model = MC_PLNPCA(q,n//16)
model.init_data(Y_sampled, O, covariates)
model.fit(20,0.001)

MSE :  tensor(1.5953, grad_fn=<MeanBackward0>)
MSE :  tensor(1.1537, grad_fn=<MeanBackward0>)
MSE :  tensor(0.7693, grad_fn=<MeanBackward0>)
MSE :  tensor(0.5224, grad_fn=<MeanBackward0>)
MSE :  tensor(0.4527, grad_fn=<MeanBackward0>)
MSE :  tensor(0.4154, grad_fn=<MeanBackward0>)
MSE :  tensor(0.4142, grad_fn=<MeanBackward0>)
MSE :  tensor(0.3931, grad_fn=<MeanBackward0>)
MSE :  tensor(0.3470, grad_fn=<MeanBackward0>)
MSE :  tensor(0.2770, grad_fn=<MeanBackward0>)
MSE :  tensor(0.2291, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1980, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1636, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1450, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1330, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1177, grad_fn=<MeanBackward0>)
likelihood :  tensor(6.7170e-14, grad_fn=<AddBackward0>)
MSE :  tensor(0.1312, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1366, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1291, grad_fn=<MeanBackward0>)
MSE :  tensor(0.1234, grad_fn=<MeanBackward0>)
MSE

MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
likelihood :  tensor(1.0977e-12, grad_fn=<AddBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE :  tensor(0.0930, grad_fn=<MeanBackward0>)
MSE

In [157]:
Y_oaks = read_csv('oaks_counts.csv').to_numpy()

2228

In [ ]:
 '''   def compute_single_log_like(self, i, acc):
        N_iter = int(1/acc)
        E = 0 
        for _ in range(N_iter): 
            W = np.random.randn(q)
            E -= 1/2*SLA.norm(W)**2
            E -= np.sum(np.exp(self.O[i,:]+self.beta.T@self.covariates[i,:]+ self.C@W)) 
            E+= np.sum((self.O[i,:]+self.beta.T@self.covariates[i,:]+ self.C@W)*self.Y[i,:])
            #print('E : ', E)
        E/= N_iter
        return E
    
    def batch_log_like(self,acc): 
        batch_E = 0
        for i in range(10): 
            batch_E += self.compute_single_log_like(i,acc) 
        return batch_E
    
    def single_grad_beta_log_like(self,i, acc): 
        N_iter = int(1/acc)
        grad = 0
        for _ in range(N_iter): 
            W = np.random.randn(q)
            grad += self.covariates[i,:].T.reshape(-1,1)@(np.exp(self.O[i,:]+ self.covariates[i,:]@self.beta+self.C@W)).reshape(1,-1)
            grad += self.covariates[i,:].T.reshape(-1,1)@(self.Y[i,:].reshape(1,-1))
        return grad/N_iter
    
    def batch_grad_beta(self, acc): 
        batch_grad = 0
        for i in range(10): 
            batch_grad += self.single_grad_beta_log_like(i,acc) 
        return batch_grad
        
   '''
        